In [1]:
import pandas as pd

In [2]:
"""import glob
data_files = glob.glob("data/*.csv")
data_files.sort()
print(len(data_files), "Files:")
print("From:", data_files[0], ", to:", data_files[-1])"""

'import glob\ndata_files = glob.glob("data/*.csv")\ndata_files.sort()\nprint(len(data_files), "Files:")\nprint("From:", data_files[0], ", to:", data_files[-1])'

In [3]:
df = pd.read_csv("data_other/merged.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# hacky temp ignore non-numerical values
df = df.drop("serial_number", axis=1)
df = df.drop("date", axis=1)
df = df.drop("model", axis=1)

In [5]:
df.fillna(0, inplace=True)

In [6]:
from tqdm import tqdm_notebook
import numpy as np

from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import cudf
import xgboost as xgb

import time

In [7]:
null_cols = df.isnull().any().tolist() 
cols = df.columns.values.tolist() 

print("Dropping columns with normalised or only 1 data:")
for col in tqdm_notebook(cols):
    current_col = df[col]
    if "norm" in col:
        df = df.drop(col, axis=1)
    elif len(current_col.unique().tolist())<2:
        df = df.drop(col, axis=1)

Dropping columns with normalised or only 1 data:


In [8]:
df = df.set_index("capacity_bytes")
df = df.drop("capacity_bytes", axis=0)
df = df.reset_index()

In [9]:
df.head()

,capacity_bytes,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,...,smart_226_raw,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw,smart_255_raw
0,4000787030016,0,192266920,0,0,10,0,131239045,0,23787,...,0,0,23536,4.31455e+10,9.41757e+10,0,0,0,0,0
1,12000138625024,0,124979220,0,0,2,0,2196252,0,825,...,0,0,104,8.30294e+08,2.7128e+08,0,0,0,0,0
2,12000138625024,0,101428480,0,0,1,0,158587645,0,1425,...,0,0,1252,1.23185e+10,1.19528e+10,0,0,0,0,0
3,4000787030016,0,0,102,436,8,0,0,42,13858,...,0,0,0,0,0,0,0,0,0,0
4,8001563222016,0,176855416,0,0,5,0,2923972399,0,11744,...,0,0,11724,4.24139e+10,7.88292e+10,0,0,0,0,0


In [10]:
df.tail()

,capacity_bytes,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,...,smart_226_raw,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw,smart_255_raw
3016675,4000787030016,0,0,100,0,3,0,0,42,20284,...,0,0,0,0,0,0,0,0,0,0
3016676,4000787030016,0,202702328,0,0,15,0,528328971,0,40255,...,0,0,40037,3.40821e+10,4.15766e+11,0,0,0,0,0
3016677,4000787030016,0,0,100,440,6,0,0,42,20117,...,0,0,0,0,0,0,0,0,0,0
3016678,4000787030016,0,0,101,490,18,0,0,42,15285,...,0,0,0,0,0,0,0,0,0,0
3016679,4000787030016,0,98538672,0,0,17,0,118410966,0,32265,...,0,0,31435,2.37899e+10,2.01715e+11,0,0,0,0,0


In [11]:
df.to_csv("cleaned.csv", sep=',', encoding='utf-8')